In [1]:
from policy import PIB, NEATProperty, NEATPolicy, NEATRequest
from cib import CIB

# Application Request

We consider an application that would like to open a new TCP connection using NEAT to a destination host `d1` with the IP `10.1.23.45`. Further, if possible, the MTU of this connection should be greater than 1500 bytes. Finally, the application specifies a `low_latency` profile.

We define three properties to represent these application requirements and combine them into a NEATRequest: 

In [2]:
property1 = NEATProperty(('remote_ip', '10.1.23.45'), precedence=NEATProperty.IMMUTABLE)
property2 = NEATProperty(('MTU', (1500, float('inf'))), precedence=NEATProperty.REQUESTED) 
property3 = NEATProperty(('transport_TCP', True))  # REQUESTED is the default property precedence
property4 = NEATProperty(('low_latency', True), precedence=NEATProperty.IMMUTABLE)

request = NEATRequest()
request.properties.insert(property1, property2, property3, property4)

print(request)
print(request.properties)

<NEATRequest: 0 candidates, 4 properties>
{(MTU|1500-inf), [low_latency|True], (transport_TCP|True), [remote_ip|10.1.23.45]}


For this scenario we map the requirement `MTU>1500` to the range `[1500, inf]`. 



# Policy Manager/NEAT logic API

Application requirements from the NEAT logic are passed to the Policy Manager using JSON. For the example above the JSON string could be:

In [3]:
request.properties.json()

'{"MTU": {"precedence": 1, "score": NaN, "value": [1500, Infinity]}, "low_latency": {"precedence": 2, "score": NaN, "value": true}, "remote_ip": {"precedence": 2, "score": NaN, "value": "10.1.23.45"}, "transport_TCP": {"precedence": 1, "score": NaN, "value": true}}'

# Exemplary Setup

Consider a host with three local interfaces `en0`, `en1`, `ra0`. Two of the interfaces, `en0`, `en1`, are wired while `ra0` is a 3G interface. We populate an instance of the Characteristic Information Base (CIB) with some information about the host interfaces and the network.

In [4]:
cib = CIB('cib/example/')

loading CIB source A.connection
loading CIB source B.connection
loading CIB source C.connection


The currently known network chracteristics are stored as entries in the CIB, where each entry contains a set of properties associated with some interface:

In [5]:
cib.dump()

A: {[MTU|9600], [local_ip|10.2.0.1], [interface|en0], [capacity|10000], <dns_name|backup.example.com>, <transport|TCP>, [is_wired|True], [remote_ip|10.1.23.45]}
B: {[MTU|1500], [interface|en1], [capacity|40000], [local_ip|192.168.1.2], <transport_UDP|True>, [is_wired|True], <interface_latency|35>, <transport_TCP|True>}
C: {[MTU|890], [interface|ra0], [remote_ip|10.1.23.45], [local_ip|10.10.2.2], [is_wired|False], [capacity|60]}


# PIB 

We create one repository for system profiles, and one for policies:

In [6]:
profiles = PIB()
pib = PIB()

For the current scenario, the low latency profile is defined as follows:

In [7]:
profile1 = NEATPolicy(name='Low latency')
profile1.match.insert(NEATProperty(('low_latency', True)))
profile1.properties.insert(NEATProperty(('iw_wired', True)))
profile1.properties.insert(NEATProperty(('interface_latency', (0,40)), precedence=NEATProperty.IMMUTABLE))

profiles.register(profile1)
profiles.dump()

===== PIB START =====
POLICY Low latency: {(low_latency|True)}  ==>  {[interface_latency|0-40], (iw_wired|True)}
===== PIB END =====


Next, we define two sample policies and add them to the Policy Information Base (PIB).

A "bulk transfer" policy is configured which is triggered by a specific destination IP, which is known to be the address of backup NFS share:

In [8]:
policy1 = NEATPolicy(name='Bulk transfer')
policy1.match.insert(NEATProperty(('remote_ip', '10.1.23.45')))
policy1.properties.insert(NEATProperty(('capacity', (10000, 100000)), precedence=NEATProperty.IMMUTABLE))
policy1.properties.insert(NEATProperty(('MTU', 9600)))

Another policy is in place to enable TCP window scaling on 10G links (if possible):

In [9]:
policy2 = NEATPolicy(name='TCP options')
policy2.match.insert(NEATProperty(('MTU', 9600)), NEATProperty(('is_wired', True)))
policy2.properties.insert(NEATProperty(('TCP_window_scale', True)))

In [10]:
pib.register(policy1)
pib.register(policy2)
pib.dump()

===== PIB START =====
POLICY Bulk transfer: {(remote_ip|10.1.23.45)}  ==>  {(MTU|9600), [capacity|10000-100000]}
POLICY TCP options: {(MTU|9600), (is_wired|True)}  ==>  {(TCP_window_scale|True)}
===== PIB END =====


# Lookup Result


## Profile Lookup

First, we apply the `low_latency` profile to the request properties. The `low_latency` property in the request is replaced by the corresponding profile properties:

In [11]:
print(request.properties)
profiles._lookup(request.properties, remove_matched=True, apply=True)
print(request.properties)

{(MTU|1500-inf), [low_latency|True], (transport_TCP|True), [remote_ip|10.1.23.45]}
{(MTU|1500-inf), (iw_wired|True), [remote_ip|10.1.23.45], [interface_latency|0-40], (transport_TCP|True)}


## CIB Lookup

Next a lookup in the CIB is performed. Our NEAT request yields three candidates:

In [12]:
cib.lookup(request)
request.dump()

{(MTU|1500-inf), (iw_wired|True), [remote_ip|10.1.23.45], [interface_latency|0-40], (transport_TCP|True)}
===== candidates =====
[0]PROPERTIES: {[MTU|1500]+1.0, [interface|en1], (iw_wired|True), [capacity|40000], [local_ip|192.168.1.2], [interface_latency|35]+1.0, [is_wired|True], [remote_ip|10.1.23.45], <transport_UDP|True>, (transport_TCP|True)+1.0}, POLICIES: set()
[1]PROPERTIES: {[MTU|9600]+1.0, <dns_name|backup.example.com>, [interface|en0], (transport_TCP|True), [capacity|10000], [local_ip|10.2.0.1], <transport|TCP>, [is_wired|True], [remote_ip|10.1.23.45]+1.0, [interface_latency|0-40], (iw_wired|True)}, POLICIES: set()
[2]PROPERTIES: {[MTU|890]-1.0, [interface|ra0], (transport_TCP|True), [remote_ip|10.1.23.45]+1.0, [local_ip|10.10.2.2], [is_wired|False], [capacity|60], [interface_latency|0-40], (iw_wired|True)}, POLICIES: set()
===== candidates =====


Each candidate is comprised of the union of the properties of a single CIB entry and the application request. Whenever  the two sets intersect, the values of the corresponding properties are compared. If two properties match, the associated candidate property score is increased (e.g., `[MTU|1500]+1.0` indicates a new score of 1.0). The score is decreased if there is a mismatch in the property values.

## PIB Lookup
In the next step the policies are applied. The "Bulk transfer" policy is applied first as it posesses the *smallest* number of match entries.

In [13]:
pib.lookup_all(request.candidates)
request.dump()

Candidate 2 is invalidated due to policy.
{(MTU|1500-inf), (iw_wired|True), [remote_ip|10.1.23.45], [interface_latency|0-40], (transport_TCP|True)}
===== candidates =====
[0]PROPERTIES: {[MTU|1500]+0.0, [interface|en1], (TCP_window_scale|True), (iw_wired|True), [capacity|40000]+1.0, [local_ip|192.168.1.2], [interface_latency|35]+1.0, [is_wired|True], [remote_ip|10.1.23.45], <transport_UDP|True>, (transport_TCP|True)+1.0}, POLICIES: {4480835312, 4480835088}
[1]PROPERTIES: {<dns_name|backup.example.com>, <transport|TCP>, [remote_ip|10.1.23.45]+1.0, [interface_latency|0-40], (TCP_window_scale|True), [MTU|9600]+2.0, [interface|en0], (iw_wired|True), [capacity|10000]+1.0, [local_ip|10.2.0.1], [is_wired|True], (transport_TCP|True)}, POLICIES: {4480835312, 4480835088}
===== candidates =====


Candidate 1 becomes:        

In [14]:
request.candidates[0].dump()

PROPERTIES: {[MTU|1500]+0.0, [interface|en1], (TCP_window_scale|True), (iw_wired|True), [capacity|40000]+1.0, [local_ip|192.168.1.2], [interface_latency|35]+1.0, [is_wired|True], [remote_ip|10.1.23.45], <transport_UDP|True>, (transport_TCP|True)+1.0}, POLICIES: {4480835312, 4480835088}


Next we examine Candidate 2:

In [15]:
request.candidates[1].dump()

PROPERTIES: {<dns_name|backup.example.com>, <transport|TCP>, [remote_ip|10.1.23.45]+1.0, [interface_latency|0-40], (TCP_window_scale|True), [MTU|9600]+2.0, [interface|en0], (iw_wired|True), [capacity|10000]+1.0, [local_ip|10.2.0.1], [is_wired|True], (transport_TCP|True)}, POLICIES: {4480835312, 4480835088}


Note that the score of the MTU property was reduced, as it did not match the requested property of the "Bulk transfer" policy.

The "TCP options" policy is not applied as the candidate does not match the policy's MTU property.

---

The third candidate was invalidated because the "Bulk transfer" policy contains an immutable property requiring a capacity of 10G, which candidate 3 cannot fulfil.

---

Finally, we can obtain the total score of the properties associated with each candidate:

In [16]:
print(request.candidates[0].score)

3.0


In [17]:
print(request.candidates[1].score)

4.0


The score indicates that candidate one (interface `en0`) is most suitable for the given application request.

# NEAT Logic
The two candidates can now be passed on to the NEAT logic as JSON strings:

In [18]:
request.candidates[0].properties.json()

'{"MTU": {"precedence": 2, "score": 0.0, "value": 1500}, "TCP_window_scale": {"precedence": 1, "score": NaN, "value": true}, "capacity": {"precedence": 2, "score": 1.0, "value": 40000}, "interface": {"precedence": 2, "score": NaN, "value": "en1"}, "interface_latency": {"precedence": 2, "score": 1.0, "value": 35}, "is_wired": {"precedence": 2, "score": NaN, "value": true}, "iw_wired": {"precedence": 1, "score": NaN, "value": true}, "local_ip": {"precedence": 2, "score": NaN, "value": "192.168.1.2"}, "remote_ip": {"precedence": 2, "score": NaN, "value": "10.1.23.45"}, "transport_TCP": {"precedence": 1, "score": 1.0, "value": true}, "transport_UDP": {"precedence": 0, "score": NaN, "value": true}}'

In [19]:
request.candidates[1].properties.json()

'{"MTU": {"precedence": 2, "score": 2.0, "value": 9600}, "TCP_window_scale": {"precedence": 1, "score": NaN, "value": true}, "capacity": {"precedence": 2, "score": 1.0, "value": 10000}, "dns_name": {"precedence": 0, "score": NaN, "value": "backup.example.com"}, "interface": {"precedence": 2, "score": NaN, "value": "en0"}, "interface_latency": {"precedence": 2, "score": NaN, "value": [0, 40]}, "is_wired": {"precedence": 2, "score": NaN, "value": true}, "iw_wired": {"precedence": 1, "score": NaN, "value": true}, "local_ip": {"precedence": 2, "score": NaN, "value": "10.2.0.1"}, "remote_ip": {"precedence": 2, "score": 1.0, "value": "10.1.23.45"}, "transport": {"precedence": 0, "score": NaN, "value": "TCP"}, "transport_TCP": {"precedence": 1, "score": NaN, "value": true}}'

Note that properties which were not matched/updated during the lookup contain `NaN` as a score. This means that the PM did not have enough information to rank theses properties. The NEAT logic must decide how to deal with these unprocessed properties.